# BO runs

In [1]:
import torch
from botorch.models import FixedNoiseGP, SingleTaskGP
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.acquisition.analytic import ExpectedImprovement
import numpy as np
import pickle
import sys
import time

load data from `prepare_Xy.ipynb`

In [2]:
X = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['X']
y = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['y']
y = np.reshape(y, (np.size(y), 1)) # for the GP
nb_data = np.size(y)
nb_data

69839

convert to torch tensors

In [3]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [4]:
X.size()

torch.Size([69839, 12])

In [5]:
y.size()

torch.Size([69839, 1])

number of COFs for initialization

In [6]:
nb_COFs_initialization = 10

In [7]:
def bo_run(nb_iterations):
    assert nb_iterations > nb_COFs_initialization
    # select initial COFs for training data randomly
    ids_acquired = np.random.choice(np.arange((nb_data)), size=nb_COFs_initialization, replace=False)

    # initialize acquired X, y
    X_acquired = X[ids_acquired, :]
    y_acquired = y[ids_acquired]
    # standardize outputs
    y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
    
    for i in range(nb_COFs_initialization, nb_iterations):
        print("iteration:", i)
        # construct and fit GP model
        model = SingleTaskGP(X_acquired, y_acquired)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_model(mll)

        # compute aquisition function at each COF in the database
        acquisition_function = ExpectedImprovement(model, best_f=y_acquired.max().item())
        acquisition_values = acquisition_function.forward(X.unsqueeze(1))

        # select COF to acquire with maximal aquisition value, which is not in the acquired set already
        ids_sorted_by_aquisition = acquisition_values.argsort(descending=True)
        for id_max_aquisition_all in ids_sorted_by_aquisition:
            if not id_max_aquisition_all.item() in ids_acquired:
                id_max_aquisition = id_max_aquisition_all.item()
                break

        # acquire this COF
        ids_acquired = np.concatenate((ids_acquired, [id_max_aquisition]))

        # update X, y acquired
        X_acquired = torch.cat([X_acquired, X[id_max_aquisition, :].unsqueeze(0)])
        y_acquired = y[ids_acquired, :] # start over to normalize y properly
        y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)

        print("\tacquired COF", id_max_aquisition, "with y = ", y[id_max_aquisition].item())
        print("\tbest y acquired:", y[ids_acquired].max().item())
    assert np.size(ids_acquired) == nb_iterations
    return ids_acquired

`ids_acquired[r, i]` will give ID of COF acquired during iteration `i` from run `r`.

In [22]:
bo_res = dict()
bo_res['nb_runs']       = 3
bo_res['nb_iterations'] = 25
bo_res['ids_acquired'] = []
ids_acquired = -np.ones((nb_runs, nb_iterations), dtype=int)
for r in range(bo_res['nb_runs']):
    print("\n\nRUN", r)
    ids_acquired = bo_run(bo_res['nb_iterations'])
    bo_res['ids_acquired'].append(ids_acquired)

with open('bo_results.pkl', 'wb') as file:
    pickle.dump(bo_res, file)



RUN 0
iteration: 10
	acquired COF 16415 with y =  174.654915912
	best y acquired: 179.569572506
iteration: 11
	acquired COF 27798 with y =  166.69585219
	best y acquired: 179.569572506
iteration: 12
	acquired COF 27035 with y =  178.57489196900002
	best y acquired: 179.569572506
iteration: 13
	acquired COF 21314 with y =  194.053101714
	best y acquired: 194.053101714
iteration: 14
	acquired COF 66263 with y =  185.76857139
	best y acquired: 194.053101714
iteration: 15
	acquired COF 33370 with y =  196.720247142
	best y acquired: 196.720247142
iteration: 16
	acquired COF 25862 with y =  167.849327414
	best y acquired: 196.720247142
iteration: 17
	acquired COF 33366 with y =  204.811726149
	best y acquired: 204.811726149
iteration: 18
	acquired COF 33330 with y =  195.58268240799998
	best y acquired: 204.811726149
iteration: 19
	acquired COF 33355 with y =  122.363855499
	best y acquired: 204.811726149
iteration: 20
	acquired COF 33332 with y =  205.963467853
	best y acquired: 205.9634